In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models

2023-12-31 00:33:25.650946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 00:33:27.516996: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/lib64
2023-12-31 00:33:27.517084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/

In [3]:
# Load a pre-trained ResNet-18 without the top layer (classifier)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modify the input layer to accept grayscale images
input_layer = layers.Input(shape=(256, 256, 1))
x = layers.Conv2D(3, (1, 1))(input_layer)  # Convert grayscale to pseudo-RGB
x = base_model(x)

# Add decoder layers for colorization
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)

x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)

x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

x = layers.Conv2D(3, (3, 3), activation='tanh', padding='same')(x)

# Create the model
model = models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 3)       6         
                                                                 
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 up_sampling2d (UpSampling2D  (None, 16, 16, 2048)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 256)       4718848   
                                                                 
 up_sampling2d_1 (UpSampling  (None, 32, 32, 256)      0         
 2D)                                                         

In [ ]:


# Load a pre-trained ResNet-18 without the top layer (classifier)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Modify the input layer to accept grayscale images
input_layer = layers.Input(shape=(256, 256, 1))
x = layers.Conv2D(3, (1, 1))(input_layer)  # Convert grayscale to pseudo-RGB
x = base_model(x)

# Add decoder layers for colorization
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)

x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)

x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

x = layers.Conv2D(3, (3, 3), activation='tanh', padding='same')(x)

# Create the model
model = models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary
model.summary()